In [1]:
# Dependencies
from sqlalchemy import create_engine
from datetime import timedelta, datetime, tzinfo
from zipfile import ZipFile
import collections
import psycopg2
import requests
import json
import pandas as pd

In [2]:
##Import the csv data

In [3]:
#data frame for legal status and pop
hr_df = pd.read_csv("../Final Project/hotelreviews.csv")


In [4]:
hr_df.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [5]:
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [6]:
###need to group by hotel, nationality, lat long,avg review score, count
##1 drop all the uneeded columns


In [13]:
#group by hotels to do a map in tableu
avg_hotel_df = hr_df.groupby(['Hotel_Name','Hotel_Address','lat','lng'])['Reviewer_Score'].mean().reset_index()
avg_hotel_df.head()



,Hotel_Name,Hotel_Address,lat,lng,Reviewer_Score
0,11 Cadogan Gardens,11 Cadogan Gardens Sloane Square Kensington an...,51.493616,-0.159235,8.845283
1,1K Hotel,13 Boulevard Du Temple 3rd arr 75003 Paris France,48.863932,2.365874,7.861486
2,25hours Hotel beim MuseumsQuartier,Lerchenfelder Stra e 1 3 07 Neubau 1070 Vienna...,48.206474,16.354630,8.983309
3,41,41 Buckingham Palace Road Westminster Borough ...,51.498147,-0.143649,9.711650
4,45 Park Lane Dorchester Collection,45 Park Lane Westminster Borough London W1K 1P...,51.506371,-0.151536,9.603571


In [15]:
#group by nationality for avg score and for lat  to do a map in tableu
avg_nationality_df = hr_df.groupby(['Reviewer_Nationality','lat','lng'])['Reviewer_Score'].mean().reset_index()
avg_nationality_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64329 entries, 0 to 64328
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Reviewer_Nationality  64329 non-null  object 
 1   lat                   64329 non-null  float64
 2   lng                   64329 non-null  float64
 3   Reviewer_Score        64329 non-null  float64
dtypes: float64(3), object(1)
memory usage: 2.0+ MB


In [11]:
#group by nationality for avg score and for lat  to do a map in tableu
counts_nationality_df = hr_df.groupby(['Reviewer_Nationality','lat','lng'])['Reviewer_Score'].count().reset_index()
counts_nationality_df.head()

,Reviewer_Nationality,lat,lng,Reviewer_Score
0,,41.328376,2.112964,1
1,,41.372784,2.172113,2
2,,41.373246,2.147435,1
3,,41.378098,2.176201,1
4,,41.379013,2.169634,2


In [10]:
#group by LAT LONG  for avg score BY LAT LONG do a map in tableu
avg_LL_df = hr_df.groupby(['lat','lng'])['Reviewer_Score'].mean().reset_index()
avg_LL_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   lat             1472 non-null   float64
 1   lng             1472 non-null   float64
 2   Reviewer_Score  1472 non-null   float64
dtypes: float64(3)
memory usage: 34.6 KB


In [16]:
###export to csv
avg_LL_df.to_csv(r'C:\Users\jcaravella\Documents\Finalproject\avg_LL_df.csv', index = False,header=True)
avg_LL_df.to_json(r'C:\Users\jcaravella\Documents\Finalproject\avg_LL_df.json')
avg_RN_df.to_csv(r'C:\Users\jcaravella\Documents\Finalproject\avg_RN_df.csv', index = False,header=True)
avg_RN_df.to_json(r'C:\Users\jcaravella\Documents\Finalproject\avg_RN_df.json')
avg_hotel_df.to_csv(r'C:\Users\jcaravella\Documents\Finalproject\avg_hotel_df.csv', index = False,header=True)
avg_hotel_df.to_json(r'C:\Users\jcaravella\Documents\Finalproject\avg_hotel_df.json')

In [11]:
####First filtor to needed indicators
#totaldeaths1 = od_df[od_df['Indicator']=='Number of Deaths']
#totaldeaths =  totaldeaths1.drop(columns=['Indicator'])
#totaldeaths.dropna()
#totaldeaths.info()


In [12]:
#rename column name
#totaldeaths = totaldeaths.rename(columns={"Data Value":"Total_Deaths"})

#totaldeaths.head()

In [47]:
############Stop unless sending to postgres###################

In [48]:
############Stop unless sending to postgres###################

In [43]:
#rds_connection_string = f"{dbuser}<insert user name>:<insert password>@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')
#password = "Hooch#12"
#dbuser = "postgres"
#rds_connection_string = f"{dbuser}:{password}@localhost:5432/Project2"
#engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
#engine.table_names()

In [47]:
#push status level to pgadmin
#Status_Counts.to_sql(name='legalstatus_level', con=engine, if_exists='replace', index=False)
#State_Counts.to_sql(name='state_level', con=engine, if_exists='replace', index=False)
